# Census Income Prediction

## Load Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load Data

In [2]:
raw_train = pd.read_csv("../data/census_income_learn.csv")
raw_test = pd.read_csv("../data/census_income_test.csv")

- Add column names
- Format target variable
- Data structure: head, info, describe
- Confirm data types
- Remove duplicates
- Target class balance
- Missing values
- Summary stats
- Numerical features
    - Summary stats
    - Histogram / KDE plots
    - Univariate analysis (filtered by target)
    - Normality tests using Shapiro-Wilk test (not needed unless we want to use ANOVA)
    - Visuals (Boxplot, distribution plot, bar chart, pearson correlation)
- Outlier values
- Skewed values?
- Drop highly correlated features
- Drop constant features
- Categorical
    - Summary stats
    - Frequency counts
    - Relationship with target
- Re-binning categorical features
    - Bin low-frequency categories into "Other" group
    - Bin similar categories together
- Statistical dependence
    - Numerical: t-test (if normal) or Mann-Whitney U test
    - Categorical: chi square + Kramer's V
- Recap initial insights


My Questions/Thoughts

- If feature contains outliers and is skewed, should I cap or transform or both?
- Features don't need to be normal if I use models that don't require normality, but I can't use certain statistical algos like linear regression and ANOVA

## Analyze Data

### Process Data

- Add column names
- Format target variable
- Data structure: head, info, describe
- Confirm data types
- Remove duplicates
- Target class balance
- Missing values

In [3]:
# List of column names

col_names = [
    "age",
    "class_of_worker",
    "detailed_industry_recode",
    "detailed_occupation_recode",
    "education",
    "wage_per_hour",
    "enroll_in_edu_inst_last_wk",
    "marital_stat",
    "major_industry_code",
    "major_occupation_code",
    "race",
    "hispanic_origin",
    "sex",
    "member_of_a_labor_union",
    "reason_for_unemployment",
    "full_or_part_time_employment_stat",
    "capital_gains",
    "capital_losses",
    "dividends_from_stocks",
    "tax_filer_stat",
    "region_of_previous_residence",
    "state_of_previous_residence",
    "detailed_household_and_family_stat",
    "detailed_household_summary_in_household",
    "instance_weight",
    "migration_code-change_in_msa",
    "migration_code-change_in_reg",
    "migration_code-move_within_reg",
    "live_in_this_house_1_year_ago",
    "migration_prev_res_in_sunbelt",
    "num_persons_worked_for_employer",
    "family_members_under_18",
    "country_of_birth_father",
    "country_of_birth_mother",
    "country_of_birth_self",
    "citizenship",
    "own_business_or_self_employed",
    "fill_inc_questionnaire_for_veteran's_admin",
    "veterans_benefits",
    "weeks_worked_in_year",
    "year",
    "income_classification"
]

In [4]:
# Assign column names to data
raw_train.columns = col_names
raw_test.columns = col_names

In [5]:
# Drop instance_weight since it should not be included in our classifiers
raw_train.drop(["instance_weight"], axis=1, inplace=True)
raw_test.drop(["instance_weight"], axis=1, inplace=True)

In [6]:
print(f"Training data target's unique values: {raw_train['income_classification'].unique()}")
print(f"Test data target's unique values: {raw_test['income_classification'].unique()}")

Training data target's unique values: [' - 50000.' ' 50000+.']
Test data target's unique values: [' - 50000.' ' 50000+.']


In [7]:
# Format target variable for training data
raw_train["target"] = np.where(
    raw_train["income_classification"] == " - 50000.",
    0,
    1
)

In [8]:
# Format target variable for test data
raw_test["target"] = np.where(
    raw_test["income_classification"] == " - 50000.",
    0,
    1
)

In [9]:
# Drop income_classification column
raw_train.drop(["income_classification"], axis=1, inplace=True)
raw_test.drop(["income_classification"], axis=1, inplace=True)

In [10]:
# Take a look at the columns
raw_train.head(5)

,age,class_of_worker,detailed_industry_recode,detailed_occupation_recode,education,wage_per_hour,enroll_in_edu_inst_last_wk,marital_stat,major_industry_code,major_occupation_code,...,country_of_birth_father,country_of_birth_mother,country_of_birth_self,citizenship,own_business_or_self_employed,fill_inc_questionnaire_for_veteran's_admin,veterans_benefits,weeks_worked_in_year,year,target
0,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,0
1,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,...,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,0
2,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,0
3,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,0
4,48,Private,40,10,Some college but no degree,1200,Not in universe,Married-civilian spouse present,Entertainment,Professional specialty,...,Philippines,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,95,0


In [11]:
print(f"The training data contains {raw_train.shape[0]} rows and {raw_train.shape[1]} columns.")
print(f"The test data contains {raw_test.shape[0]} rows and {raw_test.shape[1]} columns.")

The training data contains 199522 rows and 41 columns.
The test data contains 99761 rows and 41 columns.


In [12]:
# Ensure each column has the correct data type
raw_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199522 entries, 0 to 199521
Data columns (total 41 columns):
 #   Column                                      Non-Null Count   Dtype 
---  ------                                      --------------   ----- 
 0   age                                         199522 non-null  int64 
 1   class_of_worker                             199522 non-null  object
 2   detailed_industry_recode                    199522 non-null  int64 
 3   detailed_occupation_recode                  199522 non-null  int64 
 4   education                                   199522 non-null  object
 5   wage_per_hour                               199522 non-null  int64 
 6   enroll_in_edu_inst_last_wk                  199522 non-null  object
 7   marital_stat                                199522 non-null  object
 8   major_industry_code                         199522 non-null  object
 9   major_occupation_code                       199522 non-null  object
 10  race    

In [14]:
# Convert these objects into objects
convert_to_obj = ["detailed_industry_recode", "detailed_occupation_recode", "own_business_or_self_employed", "veterans_benefits", "year"]

raw_train[convert_to_obj] = raw_train[convert_to_obj].astype(str)
raw_test[convert_to_obj] = raw_test[convert_to_obj].astype(str)

In [ ]:
# Get rid of duplicates even in test data to avoid biasing evaluation metrics

print(f"Get rid of {raw_train.shape[0] - raw_train.drop_duplicates().shape[0]} duplicate entries in training data.")
print(f"Get rid of {raw_test.shape[0] - raw_test.drop_duplicates().shape[0]} duplicate entries in test data.")

raw_train.drop_duplicates(inplace=True)
raw_test.drop_duplicates(inplace=True)

Get rid of 46626 duplicate entries in training data.
Get rid of 20898 duplicate entries in test data.


In [ ]:
# We have a major class imbalance
raw_train["target"].value_counts(normalize=True)

target
0    0.919115
1    0.080885
Name: proportion, dtype: float64

In [ ]:
def missing_values_summary(df: pd.DataFrame, missing_value=" ?"):
    """
    Returns a DataFrame with columns for count and percentage of custom missing values,
    sorted by the highest percentage of missing values.
    
    Parameters:
        df (pd.DataFrame): The DataFrame to analyze.
        missing_value (str): The value to consider as missing (default is "?").
    
    Returns:
        pd.DataFrame: Summary of missing values.
    """
    missing_count = df.isin([missing_value]).sum()  # Count occurrences of the missing value
    total_rows = len(df)
    missing_percentage = (missing_count / total_rows)
    
    missing_summary = pd.DataFrame({
        'Missing Count': missing_count,
        'Missing Proportion': missing_percentage
    })
    
    # Filter to only include columns with missing values and sort by percentage
    missing_summary = missing_summary[missing_summary['Missing Count'] > 0]
    missing_summary = missing_summary.sort_values(by='Missing Proportion', ascending=False)
    
    return missing_summary

In [49]:
missing_values_summary(raw_train)

,Missing Count,Missing Proportion
migration_code-change_in_msa,74944,0.490163
migration_code-change_in_reg,74944,0.490163
migration_code-move_within_reg,74944,0.490163
migration_prev_res_in_sunbelt,74944,0.490163
country_of_birth_father,6383,0.041747
country_of_birth_mother,5810,0.038000
country_of_birth_self,3322,0.021727
state_of_previous_residence,698,0.004565


In [39]:
raw_train["migration_code-change_in_reg"].value_counts()


migration_code-change_in_reg
?                                 74944
Nonmover                          62204
Same county                        9479
Different county same state        2773
Different region                   1173
Different state same division       989
Abroad                              527
Different division same region      463
Not in universe                     344
Name: count, dtype: int64